In [ ]:
LIST_OF_DIRECTORIES = ["output/basfws/ambiguous/coffee", "output/basfws/ambiguous/other", "output/basfws/coffee",
                       "output/basfws/intercrop", "output/basfws/native_no_tree", "output/basfws/nativevege",
                       "output/basfws/other_tree", "output/basfws/pepper", "output/basfws/pine_trees",
                       "output/basfws/rice", "output/basfws/rubber", "output/basfws/seasonal", "output/basfws/tea",
                       "output/basfws/urban", "output/basfws/water", "output/droitefws/ambiguous/other",
                       "output/droitefws/coffee", "output/droitefws/native_no_tree", "output/droitefws/nativevege",
                       "output/droitefws/other_tree", "output/droitefws/pepper", "output/droitefws/pine_trees",
                       "output/droitefws/rice", "output/droitefws/rubber", "output/droitefws/seasonal", 
                       "output/droitefws/tea", "output/droitefws/urban", "output/droitefws/water", 
                       "output/gauchefws/ambiguous/coffee", "output/gauchefws/ambiguous/pepper",
                       "output/gauchefws/coffee", "output/gauchefws/intercrop", "output/gauchefws/native_no_tree",
                       "output/gauchefws/nativevege", "output/gauchefws/other_tree", "output/gauchefws/pepper",
                       "output/gauchefws/rice", "output/gauchefws/rubber", "output/gauchefws/seasonal",
                       "output/gauchefws/stick_pepper", "output/gauchefws/urban", "output/gauchefws/water",
                       "output/hautfws/ambiguous/coffee", "output/hautfws/ambiguous/other", "output/hautfws/cassava",
                       "output/hautfws/coffee", "output/hautfws/deciduous_forest", "output/hautfws/intercrop",
                       "output/hautfws/native_no_tree", "output/hautfws/nativevege", "output/hautfws/other_tree",
                       "output/hautfws/pepper", "output/hautfws/pine_trees", "output/hautfws/rice", 
                       "output/hautfws/rubber", "output/hautfws/seasonal", "output/hautfws/tea", "output/hautfws/urban",
                       "output/hautfws/water", "output/milieufws/cassava", "output/milieufws/coffee",
                       "output/milieufws/deciduous_forest", "output/milieufws/intercrop", 
                       "output/milieufws/native_no_tree", "output/milieufws/nativevege", "output/milieufws/other_tree",
                       "output/milieufws/pepper", "output/milieufws/rice", "output/milieufws/rubber",
                       "output/milieufws/seasonal", "output/milieufws/urban", "output/milieufws/water"]
COUNTAIN_COFFEE = "coffee"
COUNTAIN_PEPPER = "pepper"
PIXEL_LENGTH = 15
BAND_WIDTH = 12
NB_SPLIT = 5
EPOCH = 1000

# Première étape : charger les données

Importer le nom des données

In [ ]:
import os 
import numpy as np
from matplotlib import pyplot as pl
import random
import rasterio
import os
#os.environ['CUDA_VISIBLE_DEVICES']='1'

%matplotlib inline

In [ ]:
def get_all_images_names(directories): 
    images_name = []
    for directory in directories:
        counter = 0
        for filename in os.listdir(directory): 
            if os.path.isfile(directory + "/" + filename):
                images_name.append(directory + "/" + filename)
                counter += 1
            elif os.path.isdir(directory + "/" + filename):
                images_name += get_all_images_names([directory + "/" + filename])
        print(directory, counter, "images.")
    return images_name
images_name = get_all_images_names(LIST_OF_DIRECTORIES)

Importer les images et leur résultat voulu

In [ ]:
def reshape(image):
    """Reformate les données rasterio pour être utilisée par Tensorflow"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            #Fall
            reshaped_cell.append(image[0][i][j] / 65535)#Infra red
            reshaped_cell.append(image[1][i][j] / 65535)#Red
            reshaped_cell.append(image[2][i][j] / 65535)#Green
            reshaped_cell.append(image[3][i][j] / 65535)#Blue
            #Winter
            reshaped_cell.append(image[4][i][j] / 65535)#Infra red
            reshaped_cell.append(image[5][i][j] / 65535)#Red
            reshaped_cell.append(image[6][i][j] / 65535)#Green
            reshaped_cell.append(image[7][i][j] / 65535)#Blue
            #Spring
            reshaped_cell.append(image[8][i][j] / 65535)#Infra red
            reshaped_cell.append(image[9][i][j] / 65535)#Red
            reshaped_cell.append(image[10][i][j] / 65535)#Green
            reshaped_cell.append(image[11][i][j] / 65535)#Blue
            #Si les images ne possèdent pas l'intégralité des 12 bandes, utilisez plutôt le code suivant
            #for k in range(BAND_WIDTH):
            #    reshaped_cell.append(image[k][i][j] / 65535)
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
images = []
for image_name in images_name:
    if COUNTAIN_COFFEE in image_name:
        images.append((np.array(rasterio.open(image_name).read()), 2))
    elif COUNTAIN_PEPPER in image_name:
        images.append((np.array(rasterio.open(image_name).read()), 1))
    else:
        images.append((np.array(rasterio.open(image_name).read()), 0))
print("Longueur initiale : ", len(images))

In [ ]:
def add_all_rotations(images):
    result = []
    for image in images:
        result.append(image)
        for i in range (1, 4):
            rotated_image = []
            #Si les images ne possèdent pas l'intégralité des 12 bandes, utilisez BAND_WIDTH au lieu de 12
            for j in range(12):
                rotated_image.append(np.rot90(image[0][j], i))
            result.append((rotated_image, image[1]))
    return result

In [ ]:
images = add_all_rotations(images)
random.shuffle(images)

In [ ]:
images_x = []
images_y = []
for image in images:
    images_x.append(reshape(image[0]))
    images_y.append(image[1])
images_x = np.array(images_x).reshape(len(images_x), PIXEL_LENGTH, PIXEL_LENGTH, BAND_WIDTH)
images_y = np.array(images_y)
print("Longueur du set d'entrainement : ", len(images_x))

# Deuxième étape : créer notre cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics as me

In [ ]:
kfold = StratifiedKFold(n_splits=NB_SPLIT)

In [ ]:
class_weight = None
coffee = np.count_nonzero(images_y == 2)
pepper = np.count_nonzero(images_y == 1) 
other = len(images_y) - (coffee + pepper)
print("café :", str(coffee) +",", "poivre :", str(pepper) +",", "autre :", other)
if other >= coffee and other >= pepper:
    class_weight = {0: 1.,1: other / pepper, 2: other / coffee} 
elif coffee >= other and coffee >= pepper:
    class_weight = {0: coffee / other,1: coffee / pepper, 2: 1.} 
else:
    class_weight = {0: pepper / other,1: 1., 2: pepper / coffee} 

In [ ]:
correspondances = {'0':[1,0,0], '1':[0,1,0], '2':[0,0,1]}
def transform_output_value(liste):
    """StratifiedKFold n'aimant pas les valeurs de sortie en vecteur, cette fonction fera la correspondance entre 
    une valeur et un vecteur"""
    result = []
    for value in liste:
        result.append(correspondances[str(value)])
    return np.array(result)

# Troisième étape : entrainer et tester

In [ ]:
def get_fscore_recall_precision(truth, prediction, indice=2):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range(len(truth)):
        if truth[i] == indice and prediction[i] == indice:
            true_positive += 1
        elif truth[i] != indice and prediction[i] == indice:
            false_positive += 1
        elif truth[i] == indice and prediction[i] != indice:
            false_negative += 1
    precision = true_positive/(true_positive + false_positive) if (true_positive + false_positive) > 0 else None
    recall = true_positive/(true_positive + false_negative) if (true_positive + false_negative) > 0 else None
    fscore = 2 * recall * precision / (recall + precision) if recall != None and precision != None else None
    return fscore, recall, precision

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import keras
cvscores = []

coffee_avg_recall = []
coffee_avg_precision = []
coffee_avg_fscore = []

pepper_avg_recall = []
pepper_avg_precision = []
pepper_avg_fscore = []

other_avg_recall = []
other_avg_precision = []
other_avg_fscore = []

i = 0
for train, test in kfold.split(images_x, images_y):

    train_y = transform_output_value(images_y[train])
    test_y = transform_output_value(images_y[test])
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu',  input_shape=(PIXEL_LENGTH,PIXEL_LENGTH,BAND_WIDTH)))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    history = model.fit(images_x[train], train_y, epochs=EPOCH, batch_size=32, 
                        validation_data=(images_x[test], test_y), class_weight=class_weight)
    # evaluate the model
    scores = model.evaluate(images_x[test], test_y, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    model.save("model"+str(i)+"m.h5")
    i += 1
    
    pl.plot(history.history['loss'], label='Training')
    pl.plot(history.history['val_loss'], label='Testing')
    pl.ylim(0, 2) 
    pl.legend()
    pl.grid()
    print('Test score:', scores[0])
    print('Test accuracy:', scores[1])
    pl.show()
    pred = model.predict_classes(images_x[test])
    
    f, r, p = get_fscore_recall_precision(images_y[test], pred)
    print('Test recall coffee', r)
    print('Test precision coffee', p)
    print('Test f-score coffee', f)
    coffee_avg_recall.append(r * 100)
    coffee_avg_precision.append(p * 100)
    coffee_avg_fscore.append(f * 100)
    
    f, r, p = get_fscore_recall_precision(images_y[test], pred, 1)
    print('Test recall pepper', r)
    print('Test precision pepper', p)
    print('Test f-score pepper', f)
    pepper_avg_recall.append(r * 100)
    pepper_avg_precision.append(p * 100)
    pepper_avg_fscore.append(f * 100)
    
    f, r, p = get_fscore_recall_precision(images_y[test], pred, 0)
    print('Test recall other', r)
    print('Test precision other', p)
    print('Test f-score other', f)
    other_avg_recall.append(r * 100)
    other_avg_precision.append(p * 100)
    other_avg_fscore.append(f * 100)
    
    print(me.confusion_matrix(images_y[test], pred))
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

print("Recall coffee %.2f%% (+/- %.2f%%)" % (np.mean(coffee_avg_recall), np.std(coffee_avg_recall)))
print("Precision coffee %.2f%% (+/- %.2f%%)" % (np.mean(coffee_avg_precision), np.std(coffee_avg_precision)))
print("F-Score coffee %.2f%% (+/- %.2f%%)" % (np.mean(coffee_avg_fscore), np.std(coffee_avg_fscore)))

print("Recall pepper %.2f%% (+/- %.2f%%)" % (np.mean(pepper_avg_recall), np.std(pepper_avg_recall)))
print("Precision pepper %.2f%% (+/- %.2f%%)" % (np.mean(pepper_avg_precision), np.std(pepper_avg_precision)))
print("F-Score pepper %.2f%% (+/- %.2f%%)" % (np.mean(pepper_avg_fscore), np.std(pepper_avg_fscore)))

print("Recall other %.2f%% (+/- %.2f%%)" % (np.mean(other_avg_recall), np.std(other_avg_recall)))
print("Precision other %.2f%% (+/- %.2f%%)" % (np.mean(other_avg_precision), np.std(other_avg_precision)))
print("F-Score other %.2f%% (+/- %.2f%%)" % (np.mean(other_avg_fscore), np.std(other_avg_fscore)))